In [7]:
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone as p1, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader
import os

In [3]:
llm = Ollama(model="llama3")

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    
)

In [12]:
# Load PDF from the data folder
pdf_path = r'C:\Users\musta\OneDrive\Desktop\Medical Chatbot LLM\Model\data\med.pdf'
if not os.path.isfile(pdf_path):
    raise FileNotFoundError(f"File path {pdf_path} is not a valid file or url")

loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [13]:
import os
os.environ['PINECONE_API_KEY'] = '39c3b55b-2ae4-44ee-a9cd-83a99876c828'
pc = p1(
    api_key=os.environ.get("PINECONE_API_KEY")
)

In [14]:
embeding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
all_chunks = []
for document in documents:
    chunks = text_splitter.split_text(document.page_content)
    all_chunks.extend(chunks)

In [21]:
index_name="test1"
index = pc.Index("test1")  
for i, t in zip(range(len(all_chunks)), all_chunks):
   query_result = embeding.embed_query(t)
   print(i,t)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(t)} # meta data as dic
        }
    ],
    namespace="real" 
)


0 ORGANIZATIONS
American Lung Association. 1740 Broadway, New York, NY
1 10019. (800) 586-4872. <http://www.lungusa.org>.
2 Centers for Disease Control and Prevention. 1600 Clifton Rd.,
3 NE, Atlanta, GA 30333. (800) 311-3435, (404) 639-3311.<http://www.cdc.gov>.
OTHER
4 OTHER
“Occupational Lung Disease.” American Lung Association
5 Page .27 May 1998 <http://www.lungusa.org>.
6 “1996 World Surveillance Report: Selected Data Highlights.”
7 Centers for Disease Control .27 May 1998. 27 May 1998
<http://www.cdc.gov/niosh/w7_high.html>.
8 Maureen Haggerty
9 GALE ENCYCLOPEDIA OF MEDICINE 2 623ByssinosisGEM -0433 to 0624 - B  10/22/03 6:09 PM  Page 623


In [20]:
embeding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
len(documents)

637